## Final Project Submission

Please fill out:
* Student name: Christine Malinga
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Nikita Njoroge
* Blog post URL:N/A


## Import Libraries

In [1]:
import pandas as pd #to work with dataframes
import numpy as np #for any and all calculations
import sqlite3 #for working with SQL files to make dataframes

For this project, we'll be importing a variety of files to gather important data:

movieinfo: contains runtime, genres, and other details about movies.
reviews: provides ratings for movies along with written reviews, but we'll focus on the ratings only.
tmdb_movies: includes release dates and other information.
tn_movie_budgets: a key source for financial data such as production costs and revenue.
imdb_db: offers a wealth of information including ratings, titles, genres, and director details.







In [5]:
import os
import sqlite3
import pandas as pd
import zipfile

# Extract IMDb SQL .db file
with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:
    # Extract all contents of .zip file into current directory
    zipObj.extractall(path='./zippedData/')
    
# Create connection to IMDb DB
con = sqlite3.connect('./zippedData/im.db')
#import data from rt.movie_info.tsv.gz
movie_info = pd.read_csv('./zippedData/rt.movie_info.tsv.gz',sep="\t")
#rt.reviews.tsv.gz
reviews = pd.read_csv('./zippedData/rt.reviews.tsv.gz',sep="\t", encoding = 'unicode_escape')
#bom.movie_gross.csv.gz
movie_gross = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
#tmdb.movies.csv.gz
tmdb = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
#tn.movie_budgets.csv.gz
budgets = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')



### Let's begin by examining the first few files

In [10]:
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [11]:
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


### let's merge them

In [12]:
ratings_and_beyond = reviews.merge(movie_info[['rating', 'genre', 'runtime', 'id']], on = 'id', how = 'left')
ratings_and_beyond = ratings_and_beyond.merge(tmdb[['title', 'id']], on = 'id', how = 'left')
ratings_and_beyond = ratings_and_beyond.rename(columns={"id": "Id"}, errors="raise")
ratings_and_beyond

,Id,review,rating_x,fresh,critic,top_critic,publisher,date,rating_y,genre,runtime,title
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018",R,Drama|Science Fiction and Fantasy,108 minutes,NaN
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018",R,Drama|Science Fiction and Fantasy,108 minutes,NaN
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018",R,Drama|Science Fiction and Fantasy,108 minutes,NaN
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017",R,Drama|Science Fiction and Fantasy,108 minutes,NaN
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017",R,Drama|Science Fiction and Fantasy,108 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
54582,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002",R,Action and Adventure|Art House and Internation...,94 minutes,NaN
54583,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005",R,Action and Adventure|Art House and Internation...,94 minutes,NaN
54584,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005",R,Action and Adventure|Art House and Internation...,94 minutes,NaN
54585,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003",R,Action and Adventure|Art House and Internation...,94 minutes,NaN


In [ ]:
###convert date to just month, runtime to just minutes
ratings_and_beyond['date'] = pd.to_datetime(ratings_and_beyond['date'])
ratings_and_beyond['runtime'] = ratings_and_beyond['runtime'].str.replace(" minutes","")
ratings_and_beyond['runtime'] = pd.to_numeric(ratings_and_beyond['runtime'], downcast="integer")

###Nans need to be fixed

###Swap out Nans for average of the runtime column (only about 800 entries of over 54 thousand)
ratings_and_beyond['runtime'].fillna((ratings_and_beyond['runtime'].mean()), inplace=True)
ratings_and_beyond['runtime'] = ratings_and_beyond['runtime'].astype(int)

ratings_and_beyond